In [22]:
import re
from os import listdir
from collections import Counter
import pandas as pd
import itertools

In [23]:
mypath = '../luxoft/source/'
files = listdir(mypath)
words = []

for i in files:
    f = open(mypath + i, 'r', encoding='utf-8')
    t = f.read().replace('\n', ' ')
    t = t.split(' ')
    for i in t:
        i = re.sub(r'\W', ' ', i)
        i = i.split(' ')
        for j in i:
            words.append(j)
    f.close()

sorted(Counter(words).items(), key=lambda x: x[1], reverse=True)[:5]

[('', 52168),
 ('Senior', 11253),
 ('Developer', 9188),
 ('India', 7395),
 ('Engineer', 7127)]

In [24]:
def csv_df(file):
    countries = []
    headers = pd.read_csv(file, header=None)
    countries.extend(headers[headers.columns[2]].map(lambda x: x.replace('/', '')).map(lambda x: x.split(' ')))
    countries2 = []
    for i in countries:
        countries2.extend(i)
    df = pd.DataFrame(Counter(countries2).items(), columns=['tag', 'val']).sort_values('val', ascending=False)
    df['Date'] = re.findall('[0-9]{4}-[0-9]{2}-[0-9]{2}', file)[0]
    df['val'] = df['val']/df['val'].iloc[0]
    return df[['Date', 'tag', 'val']]

In [25]:
def countries():
    mypath = '../luxoft/source/'
    mean = csv_df(mypath + listdir(mypath)[0])
    mean = mean.rename(columns={'val': mean['Date'].iloc[0]})
    mean.drop('Date', axis=1, inplace=True)

    for f in listdir(mypath)[1:]:
        meanf = csv_df(mypath + f)
        meanf = meanf.rename(columns={'val': meanf['Date'].iloc[0]})
        meanf.drop('Date', axis=1, inplace=True)
        mean = pd.merge(mean, meanf, how='outer', on='tag')
        
    mean = mean.T
    mean = mean.rename(columns=mean.loc['tag'])
    mean.drop('tag', axis=0, inplace=True)
    mean.sort_index(inplace=True)
    mean_begin = mean.iloc[:4].mean()
    mean_end = mean.iloc[-4:].mean()
    change = ((mean_end - mean_begin)*100/mean_begin)
    change = change.dropna()
    change = change.astype('int64')
    x = pd.merge(mean.mean().rank().rename('mean_rank'), change.rank().rename('change_rank'), left_index=True, right_index=True)
    x = pd.merge(x, change.rename('percent'), left_index=True, right_index=True)
    x['rank'] = x['mean_rank']/3 + x['change_rank']
    change = x.sort_values('rank', ascending=False)
    x['rank1'] = (x['mean_rank'].max() - x['mean_rank'])/5 + x['change_rank']
    change1 = x.sort_values('rank1')
    num_head = int((change.count()/2)[0])
    change = pd.DataFrame({'winners': change.index[:num_head], 'increase': change['percent'].iloc[:num_head].values, \
            'losers': change1.index[:num_head], 'decrease': change1['percent'].iloc[:num_head].values}, index=[i for i in range(num_head)])
    return change.head(37)

In [26]:
countries()

C:\Users\ksn\AppData\Local\Temp\ipykernel_6864\3294330188.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_head = int((change.count()/2)[0])


,winners,increase,losers,decrease
0,India,383,Mexico,-8
1,Australia,2657,Singapore,-6
2,Kingdom,1793,States,37
3,Poland,173,Germany,40
4,Malaysia,802,Romania,99
5,Ukraine,140,Brazil,59
6,Romania,99,United,132
7,United,132,Ukraine,140
8,Serbia,166,Colombia,-27
9,Canada,302,Poland,173


In [27]:
mypath = '../luxoft/source/'
files = listdir(mypath)

for file in files:
    keywords = []
    headers = pd.read_csv(mypath + file, header=None)[1].to_list()
    for i in headers:
        i = re.sub(r'[^A-z+#]', ' ', i)
        i = re.sub(r'Data\s', 'Data_', i)
        i = re.sub(r'\sMode', '_Mode', i)
        i = re.sub(r'\sManager', '_Manager', i)
        i = re.sub(r'\sManagement', '_Management', i)
        i = re.sub(r'\sAnalyst', '_Analyst', i)
        i = re.sub(r'\sspeaker', '_speaker', i)
        i = re.sub(r'\sLearning', '_Learning', i)
        i = re.sub(r'\send', '_end', i)
        i = re.sub(r'\sStack', '_Stack', i)
        i = re.sub(r'\sLead', '_Lead', i)
        i = re.sub(r'\sengine', '_engine', i)
        i = re.sub(r'Capital\s', 'Capital_', i)
        i = i.split(' ')
        for j in i:
            if len(j) > 1 and j not in ('with', 'and', 'IT', 'to', 'in', 'for', 'or', 'experience', 'of', 'Project'):
                keywords.append(j)
                if j == 'Kingdom':
                    print(i)